In [1]:
from sklearn.model_selection import train_test_split
from transformers import TFBertTokenizer, TFBertModel
import pandas as pd
import numpy as np
import tensorflow as tf

/home/yu/kaggle-tweet-sentiment/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-26 23:28:55.403173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737952135.422076  448621 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737952135.426755  448621 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-26 23:28:55.458242: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performan

In [2]:
train = pd.read_csv("data1/train.csv").dropna()
submission = pd.read_csv("data1/test.csv").dropna()
train, test = train_test_split(train, test_size=0.15)
print(train.info())
print(test.info())
print(submission.info())
# TODO see if selected_text needs to be fixed to word boundaries

<class 'pandas.core.frame.DataFrame'>
Index: 23358 entries, 4832 to 10986
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         23358 non-null  object
 1   text           23358 non-null  object
 2   selected_text  23358 non-null  object
 3   sentiment      23358 non-null  object
dtypes: object(4)
memory usage: 912.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 4122 entries, 2611 to 10961
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         4122 non-null   object
 1   text           4122 non-null   object
 2   selected_text  4122 non-null   object
 3   sentiment      4122 non-null   object
dtypes: object(4)
memory usage: 161.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  --

In [3]:
max_text_len = 128
span_indices_size = max_text_len * (max_text_len + 1) // 2
span_indices = np.zeros((max_text_len, max_text_len), dtype=int)
span_indices[np.triu_indices(max_text_len, m=max_text_len)] = np.arange(span_indices_size)
span_indices = np.flip(span_indices, axis=1)
tokenizer = TFBertTokenizer.from_pretrained("bert-base-cased")

def tokenize(texts):
    return tokenizer(texts, padding="max_length", max_length=max_text_len)

def detokenize(ids, skip_special=True):
    return tokenizer.batch_decode(ids, skip_special_tokens=skip_special)

# https://stackoverflow.com/a/7100681
def rolling_window(a, size):
    shape = a.shape[:-1] + (a.shape[-1] - size + 1, size)
    strides = a.strides + (a. strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def find_span(texts, spans):
    texts = np.array(texts)
    spans = np.array(spans)[:, 1:]
    sizes = spans.argmin(axis=1) - 1
    span_onehot = np.zeros((texts.shape[0], span_indices_size))
    for i, text in enumerate(texts):
        window_size = sizes[i]
        matched_window = rolling_window(texts[i], window_size) == spans[i][:window_size]
        window_from = matched_window.all(axis=1).argmax()
        window_to = window_size - 1
        try:
            span_onehot[i, span_indices[window_from, window_to]] = 1
        except IndexError as e:
            print(window_from, window_to, span_indices[window_from, window_to])
            raise e
    return span_onehot

def find_y(texts, selected_texts):
    return find_span(texts["input_ids"], selected_texts["input_ids"])

def span_range(index):
    [d1], [d2] = np.where(span_indices == index)
    return d1, d1 + d2 + 1

def print_mem_info():
    info = tf.config.experimental.get_memory_info("/gpu:0")
    print(f"GPU memory usage {(info["current"] / info["peak"]):.2f}.")

tokens_train = tokenize(train.text.to_list())
tokens_test = tokenize(test.text.to_list())
selected_tokens_train = tokenize(train.selected_text.to_list())
selected_tokens_test = tokenize(test.selected_text.to_list())
y_train = find_y(tokens_train, selected_tokens_train)
y_test = find_y(tokens_test, selected_tokens_test)

tokenizer = None
print_mem_info()

2025-01-26 23:28:57.762713: I tensorflow_text/core/kernels/fast_bert_normalizer_model_builder.cc:211] CharacterSet built (lower_case_nfd_strip_accents=0). Trie data size (int32): 512. Normalized string pool size (byte): 1
I0000 00:00:1737952138.241449  448621 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21781 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


GPU memory usage 0.00.


In [4]:
batch_size = 2000
encoder = TFBertModel.from_pretrained("bert-base-cased")

def batch_inference(tokens, one=False):
    total = tokens["input_ids"].shape[0]
    stack = []
    while batch_size * len(stack) < total:
        start, end = min(batch_size, total - batch_size * len(stack)) * np.array([0, 1]) + batch_size * len(stack)
        batch = {k: v[start:end] for k, v in tokens.items()}
        outputs = encoder(batch).last_hidden_state
        with tf.device("/cpu:0"):
            stack.append(tf.identity(outputs))
        if one:
            break
    return tf.concat(stack, 0)

outputs = batch_inference(tokens_train)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [5]:
outputs.shape

TensorShape([23358, 128, 768])

In [6]:
np.save("context1", outputs[:11679].numpy())
np.save("context2", outputs[11679:].numpy())